In [1]:
import glob
import gzip
from bs4 import BeautifulSoup
import lxml
from collections import defaultdict
from tqdm import tqdm
import requests
import random
import sys
import pathlib
import csv



import argparse

# import multiprocessing
from fuzzywuzzy import fuzz

url = 'http://10.7.38.128:5200/pcse_ner_predictor?text_sentence='  # Load Balancer

result_path = '/nfs/gns/literature/machine-learning/evaluation/FP_Analysis/PCSE/'
pathlib.Path(result_path).mkdir(parents=True, exist_ok=True)



In [2]:
import pandas as pd

path_EPMC_text = '/nfs/gns/literature/machine-learning/evaluation/300articles/europePMC-NER/annotations_API/full_sentences/test_annotations/Europe_PMC_annotation.csv'
colNames = ['pmc_id', 'section', 'sentence','ner'] 
    
test_df = pd.read_csv(path_EPMC_text,sep ='\t', names=colNames)    

In [3]:
test_df_ = test_df #.head(10)
# test_df_

In [4]:
from ast import literal_eval

def deleting_epmc_GPS(list_1,del_name):
   
    for sub_list in list_1:
        if del_name in sub_list:
            list_1.remove(sub_list)
    return list_1

In [5]:
# ss = 'http://10.7.38.128:5200/pcse_ner_predictor?text_sentence=We+presented+two+cases+of+COVID-19+associated+SARS-CoV-2+infection+during+third+trimester+of+pregnancy.+Both+mothers+and+newborns+had+excellent+outcomes.+We+failed+to+identify+SARS-CoV-2+in+all+the+products+of+conception+and+the+newborns.+This+report+provided+evidence+of+low+risk+of+intrauterine+infection+by+vertical+transmission+of+SARS-CoV-2.'

# r = requests.get(ss)

# ml_json = r.json()

# for each_ml_annotation in ml_json['tagged']['entities']:
#     print(each_ml_annotation['text'])


In [6]:
from fuzzywuzzy import fuzz

def remove_FP(epmc_list, ml_json):
    all_ml_gps = []
    if ml_json['tagged']['entities']:  # ml_json['annotations']:
        for each_ml_annotation in ml_json['tagged']['entities']:# ml_json['annotations']:
            if each_ml_annotation['type'] == 'GP':
                all_ml_gps.append(each_ml_annotation['text'])
    
    non_FP_removed =[]
              

    for each_ner in epmc_list:
        if each_ner[2] == 'Gene_Proteins':
            for each_ml_gp in all_ml_gps:         
                score = fuzz.token_set_ratio(each_ml_gp, each_ner[1])
                if score >= 80:
                    non_FP_removed.append(each_ner)

    non_gp_tags =  deleting_epmc_GPS(epmc_list,'Gene_Proteins')  

    fp_removed_tags = non_gp_tags+non_FP_removed

    return fp_removed_tags


In [7]:
# from tqdm import tqdm
with open(result_path + 'pcse_fp_removal_80.tsv', 'w', newline='\n') as f1:
    public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index,row in tqdm(test_df_.iterrows(),total = len(test_df_)):
        try:
            r = requests.get(url + row['sentence'])
            if r.status_code == 200 and r.json()['status'] == 200:
                fp_removed = remove_FP(literal_eval(row['ner']), r.json())
                
        except ValueError:
            fp_removed =''
            
        public_writer.writerow([row['pmc_id'], row['section'],row['sentence'], fp_removed])   
            
    

100%|██████████| 17957/17957 [22:54<00:00, 13.07it/s]  


In [11]:
# Convert to IOB format

import sys
import os
import glob

sys.path.append('/nfs/gns/literature/Santosh_Tirunagari/test Gitlab/epmc-ml-misc-library/')

import capo_tools_lib
import evaluation_epmc_lib


iob_result_path = result_path+'iob/'
pathlib.Path(iob_result_path).mkdir(parents=True, exist_ok=True)

file_path = result_path + 'pcse_fp_removal_80.tsv'

capo_tools_lib.annotations_api_tagged_sentences_to_IOB(file_path,
                                                       iob_result_path,'pcse_fp_removal_iob_80.tsv')

100%|██████████| 17957/17957 [00:05<00:00, 3252.30it/s]


In [12]:
import metrics.ner as ner_metrics


# precision	0.7	0.7	0.72	0.73
# recall	0.53	0.53	0.54	0.55
# f1 score	0.6	0.6	0.62	0.6

#
# print(ner_metrics.semeval_scores_report(gold=epmc_labels, response=ml_labels, digits=2))

root_path = '/nfs/gns/literature/machine-learning/'
epmc_path = root_path+'Datasets/NER_Datasets/EBI_standard-IOB/test.csv'
all_tags = ['GP']

print('################ Annotation Pipeline Results ########################')
CAPO_path = iob_result_path+'pcse_fp_removal_iob_80.tsv'
for each_tag in all_tags:
    print('############ '+each_tag+' ####################')
    print('\n')
    print(ner_metrics.semeval_report(gold_path=epmc_path, response_path=CAPO_path, targets=[each_tag]))

################ Annotation Pipeline Results ########################
############ GP ####################


               strict     exact   partial      type

   correct      3,055     3,055     3,055     3,214
 incorrect        159       159         0         0
   partial          0         0       159         0
   missing      3,354     3,354     3,354     3,354
  spurious        319       319       319       319
 precision       0.86      0.86      0.89      0.91
    recall       0.47      0.47      0.48      0.49
  f1 score       0.60      0.60      0.62      0.64
Gold Total      6,566
Resp Total      3,533


